In [1]:
import cv2
import threading   
import playsound   
import smtplib     

In [2]:
fire_cascade = cv2.CascadeClassifier('fire-cascade.xml')

In [3]:
# Use "0" for laptop inbuilt camera and "1" for USB attached camera or webcam
# vid = cv2.VideoCapture(0) 

vid = cv2.VideoCapture("videos\\fire2.mp4")
runOnce = False # created boolean

In [4]:
# Alarm is played after fire detection

def play_alarm_sound_function(): 
    playsound.playsound('fire-alarm.mp3',True) 
    print("Fire alarm end") # for testing

In [5]:
# Mail is sent after fire detection

def send_mail_function(): 
    
    recipientmail = "johndoe@gmail.com" # recipients mail
    recipientmail = recipientmail.lower()
    
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.ehlo()
        server.starttls()
        server.login("asender email", 'sender password') # Logs using sender's credentials
        server.sendmail('add recipients mail', recipientmail, "REPORT! FIRE ACCIDENT DETECTED") 
        print("Alert mail sent sucesfully to {}".format(recipientmail)) # for testing
        server.close()
        
    except Exception as e:
        print(e)
		
while(True):
    Alarm_Status = False
    ret, frame = vid.read() 
    # Checks for empty frame, always prints during tests without video input
    if not ret:
        print("Failed to grab frame")
    break
 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    fire = fire_cascade.detectMultiScale(frame, 1.2, 5) 

    ## to highlight fire with square 
    for (x,y,w,h) in fire:
        cv2.rectangle(frame,(x-20,y-20),(x+w+20,y+h+20),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        print("Fire alarm initiated")
        # To call alarm thread
        threading.Thread(target=play_alarm_sound_function).start()  

        if runOnce == False:
            print("Mail send initiated")
            # To call alarm thread
            threading.Thread(target=send_mail_function).start() 
            runOnce = True
        if runOnce == True:
            print("Mail is already sent once")
            runOnce = True

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

Failed to grab frame
